In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/content/drive/MyDrive/Sorachio-360M-Chat/models"
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_path)

questions = [
    "Who are you?",
    "Who created you?",
    "Are you from OpenAI?",
    "What's your purpose?",
    "Do you work like ChatGPT?",
    "What makes you different?",
    "Tell me about Sorachio.",
    "What is the capital of United States?",
    "Who is Donald Trump?",
    "What do you know about Izzul Fahmi?"
]

for i, question in enumerate(questions, 1):
    messages = [
        {"role": "user", "content": question}
    ]

    chat_input = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(chat_input, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.6,
        top_p=0.85,
        repetition_penalty=1.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)

    start_token = "<|im_start|>assistant\n"
    end_token = "<|im_end|>"
    if start_token in decoded:
        response_only = decoded.split(start_token)[-1].split(end_token)[0].strip()
    else:
        response_only = decoded.strip()

    print(f"Q{i}: {question}")
    print(f"A{i}: {response_only}")
    print("=" * 50)
